In [8]:
# The natural language toolkit for tokenizing words and sentences
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
#scikit-learn is a key data science module like pandas
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import os
import re
import config5
import pandas as pd
from string import punctuation
from datetime import datetime

DIRNAME = 'blogs/'

english_stopwords = set(stopwords.words('english')+list(punctuation)+['..','...','....','``','//n',"''"])


{'!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 "''",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '..',
 '...',
 '....',
 '/',
 '//n',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 '``',
 'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',


In [23]:
def clean_word_list(filelist):
    '''
        Takes a list of XML files
    '''
    corpus = ''
    lemmatizer = WordNetLemmatizer()
    for eachfile in filelist:
        xmltext = open(eachfile, encoding='utf8',errors='replace').read()
        soup = BeautifulSoup(xmltext,features='xml')
        corpus += '/n'.join([x.text.lower() for x in soup.findAll('post')])
    wordlist = [lemmatizer.lemmatize(x) for x in word_tokenize(corpus) if x not in english_stopwords]
    return wordlist

In [18]:
blogfiles = [ DIRNAME + fn for fn in os.listdir(DIRNAME)[:2000] if not fn.startswith('.')]
teenblog = [1 if re.search(r'\.1[3-9]\.',x) else 0 for x in blogfiles] 

In [19]:
list(zip(blog_list, teenblog))[:25]

[('blogs/1000331.female.37.indUnk.Leo.xml', 0),
 ('blogs/1000866.female.17.Student.Libra.xml', 1),
 ('blogs/1004904.male.23.Arts.Capricorn.xml', 0),
 ('blogs/1005076.female.25.Arts.Cancer.xml', 0),
 ('blogs/1005545.male.25.Engineering.Sagittarius.xml', 0),
 ('blogs/1007188.male.48.Religion.Libra.xml', 0),
 ('blogs/100812.female.26.Architecture.Aries.xml', 0),
 ('blogs/1008329.female.16.Student.Pisces.xml', 1),
 ('blogs/1009572.male.25.indUnk.Cancer.xml', 0),
 ('blogs/1011153.female.27.Technology.Virgo.xml', 0),
 ('blogs/1011289.female.25.indUnk.Libra.xml', 0),
 ('blogs/1011311.female.17.indUnk.Scorpio.xml', 1),
 ('blogs/1013637.male.17.RealEstate.Virgo.xml', 1),
 ('blogs/1015252.female.23.indUnk.Pisces.xml', 0),
 ('blogs/1015556.male.34.Technology.Virgo.xml', 0),
 ('blogs/1016560.male.41.Publishing.Sagittarius.xml', 0),
 ('blogs/1016738.male.26.Publishing.Libra.xml', 0),
 ('blogs/1016787.female.24.Communications-Media.Leo.xml', 0),
 ('blogs/1019224.female.27.RealEstate.Libra.xml', 0),


In [24]:
print(datetime.now().strftime(':%M:%S'))
allblogs = [' '.join(clean_word_list([x])) for x in blogfiles]
print(datetime.now().strftime(':%M:%S'))


:49:14
:54:58


In [25]:
df = pd.DataFrame({'filename':blogfiles,'teen':teenblog,'text':allblogs})

In [26]:
# x is the text in the post and y is our is teen value and this is exploring whether features link these
# splts them and tries out fetrue identificationa dn then feature applciation on control
# we could say what split to use, usually 50/50
x_train, x_test, y_train, y_test = train_test_split(df.text, df.teen, random_state=1)

In [29]:
tfidf_vectorizer = TfidfVectorizer(use_idf=True)
# getting a whole set of vectors with words and their frequency in documents
x_train_tf = tfidf_vectorizer.fit_transform(x_train)
# this is our second half from the split that we are preparing to use for prediction with naive bayes
x_test_tf = tfidf_vectorizer.transform(x_test)

In [31]:
naive_bayes = MultinomialNB()
# ask NB to try fitting this with our training data fo our vectors and the 1/0 for teen or not
# check if there is a likelihood of these features map to whteher we have a teen blog
naive_bayes.fit(x_train_tf, y_train)
# now try to predict for the second half to see if you can get teen status right
predictions = naive_bayes.predict(x_test_tf)

In [33]:
print('Accuracy:',accuracy_score(y_test, predictions))
# NOT a good prediction using naive bayes here

Accuracy: 0.612


In [5]:
#df.head()
corpus = '\n'.join(x.strip() for x in df.text)
corpus = corpus[0:100000] # shrinking the massive corpus for now
#for blogtext in df.text:
#    all_blogtext.append(blogtext.strip())



print(corpus[0:90000])
# we could sue corpus.split() but that does nto handle things like words connected by elipsis well
#corpus_sentence_tokenized = sent_tokenize(corpus)
#corpus_sentence_tokenized

Info has been found (+/- 100 pages, and 4.5 MB of .pdf files) Now i have to wait untill our team leader has processed it and learns html.
These are the team members:   Drewes van der Laag           urlLink mail  Ruiyu Xie                     urlLink mail  Bryan Aaldering (me)          urlLink mail
In het kader van kernfusie op aarde:  MAAK JE EIGEN WATERSTOFBOM   How to build an H-Bomb From: ascott@tartarus.uwa.edu.au (Andrew Scott) Newsgroups: rec.humor Subject: How To Build An H-Bomb (humorous!) Date: 7 Feb 1994 07:41:14 GMT Organization: The University of Western Australia  Original file dated 12th November 1990. Seemed to be a transcript of a 'Seven Days' article. Poorly formatted and corrupted. I have added the text between 'examine under a microscope' and 'malleable, like gold,' as it was missing. If anyone has the full text, please distribute. I am not responsible for the accuracy of this information. Converted to HTML by Dionisio@InfiNet.com 11/13/98. (Did a little spell-checki

In [15]:
print(corpus[0:90000])

Info has been found (+/- 100 pages, and 4.5 MB of .pdf files) Now i have to wait untill our team leader has processed it and learns html.
These are the team members:   Drewes van der Laag           urlLink mail  Ruiyu Xie                     urlLink mail  Bryan Aaldering (me)          urlLink mail
In het kader van kernfusie op aarde:  MAAK JE EIGEN WATERSTOFBOM   How to build an H-Bomb From: ascott@tartarus.uwa.edu.au (Andrew Scott) Newsgroups: rec.humor Subject: How To Build An H-Bomb (humorous!) Date: 7 Feb 1994 07:41:14 GMT Organization: The University of Western Australia  Original file dated 12th November 1990. Seemed to be a transcript of a 'Seven Days' article. Poorly formatted and corrupted. I have added the text between 'examine under a microscope' and 'malleable, like gold,' as it was missing. If anyone has the full text, please distribute. I am not responsible for the accuracy of this information. Converted to HTML by Dionisio@InfiNet.com 11/13/98. (Did a little spell-checki

In [6]:
#print(corpus[0:90000])
# we could sue corpus.split() but that does nto handle things like words connected by elipsis well
corpus_word_tokenized = word_tokenize(corpus[:90000])

#does a greta job of pulling words away from punctuation, etc
corpus_word_tokenized[0:100]

['Info',
 'has',
 'been',
 'found',
 '(',
 '+/-',
 '100',
 'pages',
 ',',
 'and',
 '4.5',
 'MB',
 'of',
 '.pdf',
 'files',
 ')',
 'Now',
 'i',
 'have',
 'to',
 'wait',
 'untill',
 'our',
 'team',
 'leader',
 'has',
 'processed',
 'it',
 'and',
 'learns',
 'html',
 '.',
 'These',
 'are',
 'the',
 'team',
 'members',
 ':',
 'Drewes',
 'van',
 'der',
 'Laag',
 'urlLink',
 'mail',
 'Ruiyu',
 'Xie',
 'urlLink',
 'mail',
 'Bryan',
 'Aaldering',
 '(',
 'me',
 ')',
 'urlLink',
 'mail',
 'In',
 'het',
 'kader',
 'van',
 'kernfusie',
 'op',
 'aarde',
 ':',
 'MAAK',
 'JE',
 'EIGEN',
 'WATERSTOFBOM',
 'How',
 'to',
 'build',
 'an',
 'H-Bomb',
 'From',
 ':',
 'ascott',
 '@',
 'tartarus.uwa.edu.au',
 '(',
 'Andrew',
 'Scott',
 ')',
 'Newsgroups',
 ':',
 'rec.humor',
 'Subject',
 ':',
 'How',
 'To',
 'Build',
 'An',
 'H-Bomb',
 '(',
 'humorous',
 '!',
 ')',
 'Date',
 ':',
 '7',
 'Feb',
 '1994']

In [3]:
import nltk
#nltk.download('wordnet')

english_stopwords = set(stopwords.words('english')+list(punctuation))
english_stopwords

{'!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'need

In [21]:
stemmer = PorterStemmer()
stemmed = [stemmer.stem(x) for x in corpus_word_tokenized[:1000]]

# zip makes the two arrays into tuples so we can see what changed
list(zip(corpus_word_tokenized[:1000], stemmed))

[('Info', 'info'),
 ('has', 'ha'),
 ('been', 'been'),
 ('found', 'found'),
 ('(', '('),
 ('+/-', '+/-'),
 ('100', '100'),
 ('pages', 'page'),
 (',', ','),
 ('and', 'and'),
 ('4.5', '4.5'),
 ('MB', 'mb'),
 ('of', 'of'),
 ('.pdf', '.pdf'),
 ('files', 'file'),
 (')', ')'),
 ('Now', 'now'),
 ('i', 'i'),
 ('have', 'have'),
 ('to', 'to'),
 ('wait', 'wait'),
 ('untill', 'until'),
 ('our', 'our'),
 ('team', 'team'),
 ('leader', 'leader'),
 ('has', 'ha'),
 ('processed', 'process'),
 ('it', 'it'),
 ('and', 'and'),
 ('learns', 'learn'),
 ('html', 'html'),
 ('.', '.'),
 ('These', 'these'),
 ('are', 'are'),
 ('the', 'the'),
 ('team', 'team'),
 ('members', 'member'),
 (':', ':'),
 ('Drewes', 'drew'),
 ('van', 'van'),
 ('der', 'der'),
 ('Laag', 'laag'),
 ('urlLink', 'urllink'),
 ('mail', 'mail'),
 ('Ruiyu', 'ruiyu'),
 ('Xie', 'xie'),
 ('urlLink', 'urllink'),
 ('mail', 'mail'),
 ('Bryan', 'bryan'),
 ('Aaldering', 'aalder'),
 ('(', '('),
 ('me', 'me'),
 (')', ')'),
 ('urlLink', 'urllink'),
 ('mail', 'm

In [27]:
# moving from stemming to lemmatization (which is better)
sample = "This is a sentence.  Also, this here is another sentence"


# ^ this is to set up the parts of speech mechanism
pos_tagged = [nltk.pos_tag(word_tokenize(sent)) for sent in sent_tokenize(sample)]
pos_tagged

# this helps us identify how a word is being used (noun, verb, etc for one word)

[[('This', 'DT'), ('is', 'VBZ'), ('a', 'DT'), ('sentence', 'NN'), ('.', '.')],
 [('Also', 'RB'),
  (',', ','),
  ('this', 'DT'),
  ('here', 'RB'),
  ('is', 'VBZ'),
  ('another', 'DT'),
  ('sentence', 'NN')]]

In [14]:
# lemmatizer understands the language and does not just change words by rule
lemmatizer = WordNetLemmatizer()
lemm = [lemmatizer.lemmatize(x) for x in corpus_word_tokenized[:1000] if x not in english_stopwords]
lemm

new_corpus=""
for word in lemm:
    new_corpus += ' '+word
new_corpus

" Info found +/- 100 page 4.5 MB .pdf file Now wait untill team leader processed learns html These team member Drewes van der Laag urlLink mail Ruiyu Xie urlLink mail Bryan Aaldering urlLink mail In het kader van kernfusie op aarde MAAK JE EIGEN WATERSTOFBOM How build H-Bomb From ascott tartarus.uwa.edu.au Andrew Scott Newsgroups rec.humor Subject How To Build An H-Bomb humorous Date 7 Feb 1994 07:41:14 GMT Organization The University Western Australia Original file dated 12th November 1990 Seemed transcript 'Seven Days article Poorly formatted corrupted I added text 'examine microscope 'malleable like gold missing If anyone full text please distribute I responsible accuracy information Converted HTML Dionisio InfiNet.com 11/13/98 Did little spell-checking minor edits Stolen urlLink http //my.ohio.voyager.net/~dionisio/fun/m ... own-h-bomb.html reformatted HTML It validates XHTML 1.0 Strict How Build H-Bomb Making owning H-bomb kind challenge real Americans seek Who want passive victim

In [17]:
tfidf_vectorizer = TfidfVectorizer(use_idf=True)
tfidf_vectors = tfidf_vectorizer.fit_transform(new_corpus)
tfidf_vectors

ValueError: Iterable over raw text documents expected, string object received.

In [19]:
from string import punctuation
stopped_corpus = [x for x in corpus if not x in english_stopwords]



['I',
 'n',
 'f',
 ' ',
 'h',
 ' ',
 'b',
 'e',
 'e',
 'n',
 ' ',
 'f',
 'u',
 'n',
 ' ',
 '(',
 '+',
 '/',
 '-',
 ' ',
 '1',
 '0',
 '0',
 ' ',
 'p',
 'g',
 'e',
 ',',
 ' ',
 'n',
 ' ',
 '4',
 '.',
 '5',
 ' ',
 'M',
 'B',
 ' ',
 'f',
 ' ',
 '.',
 'p',
 'f',
 ' ',
 'f',
 'l',
 'e',
 ')',
 ' ',
 'N',
 'w',
 ' ',
 ' ',
 'h',
 'v',
 'e',
 ' ',
 ' ',
 'w',
 ' ',
 'u',
 'n',
 'l',
 'l',
 ' ',
 'u',
 'r',
 ' ',
 'e',
 ' ',
 'l',
 'e',
 'e',
 'r',
 ' ',
 'h',
 ' ',
 'p',
 'r',
 'c',
 'e',
 'e',
 ' ',
 ' ',
 'n',
 ' ',
 'l',
 'e',
 'r',
 'n',
 ' ',
 'h',
 'l',
 '.',
 '\n',
 'T',
 'h',
 'e',
 'e',
 ' ',
 'r',
 'e',
 ' ',
 'h',
 'e',
 ' ',
 'e',
 ' ',
 'e',
 'b',
 'e',
 'r',
 ':',
 ' ',
 ' ',
 ' ',
 'D',
 'r',
 'e',
 'w',
 'e',
 ' ',
 'v',
 'n',
 ' ',
 'e',
 'r',
 ' ',
 'L',
 'g',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 'u',
 'r',
 'l',
 'L',
 'n',
 'k',
 ' ',
 'l',
 ' ',
 ' ',
 'R',
 'u',
 'u',
 ' ',
 'X',
 'e',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' 

In [20]:
stopped_corpus = [x for x in corpus if not x in punctuation]
stopped_corpus

['I',
 'n',
 'f',
 'o',
 ' ',
 'h',
 'a',
 's',
 ' ',
 'b',
 'e',
 'e',
 'n',
 ' ',
 'f',
 'o',
 'u',
 'n',
 'd',
 ' ',
 ' ',
 '1',
 '0',
 '0',
 ' ',
 'p',
 'a',
 'g',
 'e',
 's',
 ' ',
 'a',
 'n',
 'd',
 ' ',
 '4',
 '5',
 ' ',
 'M',
 'B',
 ' ',
 'o',
 'f',
 ' ',
 'p',
 'd',
 'f',
 ' ',
 'f',
 'i',
 'l',
 'e',
 's',
 ' ',
 'N',
 'o',
 'w',
 ' ',
 'i',
 ' ',
 'h',
 'a',
 'v',
 'e',
 ' ',
 't',
 'o',
 ' ',
 'w',
 'a',
 'i',
 't',
 ' ',
 'u',
 'n',
 't',
 'i',
 'l',
 'l',
 ' ',
 'o',
 'u',
 'r',
 ' ',
 't',
 'e',
 'a',
 'm',
 ' ',
 'l',
 'e',
 'a',
 'd',
 'e',
 'r',
 ' ',
 'h',
 'a',
 's',
 ' ',
 'p',
 'r',
 'o',
 'c',
 'e',
 's',
 's',
 'e',
 'd',
 ' ',
 'i',
 't',
 ' ',
 'a',
 'n',
 'd',
 ' ',
 'l',
 'e',
 'a',
 'r',
 'n',
 's',
 ' ',
 'h',
 't',
 'm',
 'l',
 '\n',
 'T',
 'h',
 'e',
 's',
 'e',
 ' ',
 'a',
 'r',
 'e',
 ' ',
 't',
 'h',
 'e',
 ' ',
 't',
 'e',
 'a',
 'm',
 ' ',
 'm',
 'e',
 'm',
 'b',
 'e',
 'r',
 's',
 ' ',
 ' ',
 ' ',
 'D',
 'r',
 'e',
 'w',
 'e',
 's',
 ' ',
 'v',
 'a

In [35]:
numbers = [1,2,3,4,5,6,7]

choice = ['yes' if x%3>0 else 'no' for x in numbers ]
choice

['yes', 'yes', 'no', 'yes', 'yes', 'no', 'yes']